<a href="https://colab.research.google.com/github/Tanzhengtong/Genre_Classification_Based_Bert_with_fine_tune/blob/main/Bert_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
import json
import numpy as np
import pandas as pd
import time
import random
import torch
import torch.nn as nn

from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from string import punctuation
from gensim.models import Word2Vec
import re
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from sklearn.metrics import roc_auc_score,f1_score
# from torchtext import data
from torchtext.legacy import data
import spacy
import datetime

from torch.autograd import Variable
import torch.nn.functional as functional
from transformers import BertForSequenceClassification, AdamW, BertConfig
import gc
from transformers import BertModel
import time

import nltk
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
nltk.download('stopwords')

     |████████████████████████████████| 2.8 MB 9.6 MB/s 
     |████████████████████████████████| 895 kB 64.2 MB/s 
     |████████████████████████████████| 636 kB 73.1 MB/s 
     |████████████████████████████████| 3.3 MB 67.8 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# load the training data
train_data = json.load(open("/content/drive/MyDrive/Colab Notebooks/genre_train.json", "r"))
X = train_data['X']
Y = train_data['Y'] # id mapping: 0 - Horror, 1 - Science Fiction, 2 - Humor, 3 - Crime Fiction
docid = train_data['docid'] # these are the ids of the books which each training example came from

In [ ]:
def preprocess(data):
    #remove url and hashtag
    for i in range(len(data)):
        text=data[i].lower()
        text1=''.join([word+" " for word in text.split()])
        data[i]=text1
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    space_pattern = '\s+'

    for i in range(len(data)):
        text_string = data[i]
        parsed_text = re.sub(hashtag_regex, '', text_string)
        parsed_text = re.sub(giant_url_regex, '', parsed_text)
        parsed_text = re.sub(mention_regex, '', parsed_text) 
        #remove punctuation
        parsed_text = re.sub(r"[{}]+".format(punctuation), '', parsed_text) 
        parsed_text = re.sub(space_pattern, ' ', parsed_text)
        data[i] = parsed_text
    return data
content_clean = preprocess(X)


In [ ]:
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Print the original sentence.
print(' Original: ', content_clean[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(content_clean[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(content_clean[0])))

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

 Original:  go right in there he gaped at my arm when the bandages were off i took a quick glance and wished i hadnt how did you do this smoking in bed i said have you got something that he caught me before i hit the floor got me into a chair then he had that scotch hed been wanting gave me a shot as an afterthought and looked at me narrowly i suppose you fell out of that same bed and broke your leg he said right hell of a dangerous bed 
Tokenized:  ['go', 'right', 'in', 'there', 'he', 'gaped', 'at', 'my', 'arm', 'when', 'the', 'bandages', 'were', 'off', 'i', 'took', 'a', 'quick', 'glance', 'and', 'wished', 'i', 'hadn', '##t', 'how', 'did', 'you', 'do', 'this', 'smoking', 'in', 'bed', 'i', 'said', 'have', 'you', 'got', 'something', 'that', 'he', 'caught', 'me', 'before', 'i', 'hit', 'the', 'floor', 'got', 'me', 'into', 'a', 'chair', 'then', 'he', 'had', 'that', 'scotch', 'he', '##d', 'been', 'wanting', 'gave', 'me', 'a', 'shot', 'as', 'an', 'after', '##th', '##ough', '##t', 'and', 'loo

In [ ]:
input_ids = []
attention_masks = []
for content in content_clean:
    encoded_dict = tokenizer.encode_plus(
                        content,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(Y)

# Print sentence 0, now as a list of IDs.
print('Original: ', content_clean[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  go right in there he gaped at my arm when the bandages were off i took a quick glance and wished i hadnt how did you do this smoking in bed i said have you got something that he caught me before i hit the floor got me into a chair then he had that scotch hed been wanting gave me a shot as an afterthought and looked at me narrowly i suppose you fell out of that same bed and broke your leg he said right hell of a dangerous bed 
Token IDs: tensor([  101,  2175,  2157,  1999,  2045,  2002, 23832,  2012,  2026,  2849,
         2043,  1996, 27993,  2020,  2125,  1045,  2165,  1037,  4248,  6054,
         1998,  6257,  1045,  2910,  2102,  2129,  2106,  2017,  2079,  2023,
         9422,  1999,  2793,  1045,  2056,  2031,  2017,  2288,  2242,  2008,
         2002,  3236,  2033,  2077,  1045,  2718,  1996,  2723,  2288,  2033,
         2046,  1037,  3242,  2059,  2002,  2018,  2008, 18937,  2002,  2094,
         2042,  5782,  2435,  2033,  1037,  2915,  2004,  2019,  2044,  2705,
  

In [ ]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size],generator=torch.Generator().manual_seed(42))

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

6,068 training samples
  675 validation samples


In [ ]:
batch_size =16

train_dataloader = DataLoader(
            train_dataset,  
            shuffle = True,
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset,
            shuffle = False,
            batch_size = batch_size 
        )

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 4, 
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# with torch.no_grad():
torch.cuda.empty_cache()

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )
epochs = 3
criterion = nn.CrossEntropyLoss()
seed_val = 27

random.seed(seed_val)
torch.manual_seed(seed_val)

training_stats = []
total_t0 = time.time()
best_accuracy = 0
for epoch_i in range(0, epochs):
    #Training
    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    total_train_accuracy = 0
    model.train()
    for step, batch in enumerate(train_dataloader):

        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)
        model.zero_grad()        
        out = model(input_ids, token_type_ids=None, attention_mask=input_mask, labels=labels)
        loss = out[0]
        logits = out[1]
        
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        pred = torch.argmax(logits, dim = 1)
        total_train_accuracy +=  torch.sum(pred == labels).item()
        
    avg_train_accuracy = total_train_accuracy / len(train_dataloader.dataset)
    avg_train_loss = total_train_loss / len(train_dataloader.dataset)            
    print("  Accuracy: {}".format(avg_train_accuracy))
    print("  Training loss: {}".format(avg_train_loss))
    
    # Validation
    print("")
    print("Validation...")
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    y_true = []
    y_pred = []

    for batch in validation_dataloader:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():        
            out = model(input_ids, token_type_ids=None, attention_mask=input_mask,labels=labels)
            loss = out[0]
            logits = out[1]

        total_eval_loss += loss.item()
        pred = torch.argmax(logits, dim = 1)
        total_eval_accuracy += torch.sum(pred == labels).item()
        y_true.append(labels.flatten())
        y_pred.append(pred.flatten())
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader.dataset)
    print("  Accuracy: {}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader.dataset)
    print("  Validation loss: {}".format(avg_val_loss))
    training_time = format_time(time.time() - t0)
    print()
    
    y_true = torch.cat(y_true).tolist()
    y_pred = torch.cat(y_pred).tolist()
    print("This epoch took: {:}".format(training_time))
    # print('roc_auc score: ', roc_auc_score(y_true,y_pred))
    # print('F1 score:',f1_score(y_true, y_pred))
    print()
    
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Train Accur.': avg_train_accuracy,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
        }
    )
    print()

    if avg_val_accuracy > best_accuracy:
        best_accuracy = avg_val_accuracy
        best_model = model

print()
print("="*10)
print("Summary")
print("Total time {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

NameError: ignored

# testing

In [ ]:
# load the test data
# the test data does not have labels, our model needs to generate these
test_data = json.load(open("/content/drive/MyDrive/Colab Notebooks/genre_test.json", "r"))
Xt = test_data['X']
content_clean_test = preprocess(Xt)


print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
input_ids_test = []
attention_masks_test = []
for tweet in content_clean_test:
    encoded_dict = tokenizer.encode_plus(
                        tweet,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_test.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_test.append(encoded_dict['attention_mask'])
# Convert the lists into tensors.
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)
# Print sentence 0, now as a list of IDs.
print('Original: ', content_clean_test[0])
print('Token IDs:', input_ids_test[0])



# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids_test, attention_masks_test)

batch_size = 256
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
test_dataloader = DataLoader(
            dataset,  
            shuffle = False,
            batch_size = batch_size 
        )

Loading BERT tokenizer...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  it was too awful i never thought never for a moment that my poor dear mother was buried in the crypt and when i went to look at the name on the coffin that was nearest to where i was i knocked away the dust and then i saw her name margaret norman aetat 22 i couldnt bear it she was only a girl herself only just twice my agelying there in that terrible dark place with all the thick dust and the spiders webs oh harold harold how shall i ever bear to think of her lying there and that i shall never see her dear face never never he tried to soothe her by patting and holding her hands 
Token IDs: tensor([  101,  2009,  2001,  2205,  9643,  1045,  2196,  2245,  2196,  2005,
         1037,  2617,  2008,  2026,  3532,  6203,  2388,  2001,  3950,  1999,
         1996, 19888,  1998,  2043,  1045,  2253,  2000,  2298,  2012,  1996,
         2171,  2006,  1996, 13123,  2008,  2001,  7205,  2000,  2073,  1045,
         2001,  1045,  6573,  2185,  1996,  6497,  1998,  2059,  1045,  2387,
  

In [ ]:
# Make prediction
model.eval()
result=np.array
for step, batch in enumerate(test_dataloader):
  with torch.no_grad():
     id=batch[0].to(device)
     mask=batch[1].to(device)
     raw_pred = model(id,mask)
  logits = raw_pred[0]
  logits = logits.detach().cpu().numpy()
  pred = np.argmax(logits, axis = 1)
  # print(step)
  result=np.concatenate((result, pred), axis=None)

test_result=result[1:].tolist()
fout = open("/content/drive/MyDrive/Colab Notebooks/out-0927-v1.csv", "w")
fout.write("Id,Y\n")
for i, line in enumerate(test_result): # Y_test_pred is in the same order as the test data
    fout.write("%d,%d\n" % (i, line))
fout.close()


In [ ]:
# from google.colab import files
# torch.save(model.state_dict(), 'checkpoint.pth')


# # download checkpoint file
# files.download('checkpoint.pth')